# BERT part of speech tagger

In [1]:
import torch
# If GPU available
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 3GB


In [2]:
from batchify import *
from create_vocab import *
from data_handling import *
from parser import *
from projectivize import *
from uas import *
from window_models import *
from taggers import *

## Import data

In [3]:
train_data = Dataset('data/en_gum-ud-train-projectivized.conllu')
dev_data = Dataset('data/en_gum-ud-dev-projectivized.conllu')
test_data = Dataset('data/en_gum-ud-test-projectivized.conllu')

In [4]:
len(list([word for elem in train_data for word in elem]))

86148

In [18]:
word_vocab, label_vocab = make_vocabs(train_data)

In [111]:
words = [[word[0] for word in sent] for sent in train_data]
tags = [word[1] for sent in train_data for word in sent]

In [142]:
#words[:10]

## Load embeddings

In [1]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('path/to/file')
weights = torch.FloatTensor(model.vectors) 
embedding = from_pretrained(weights)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/file'

In [ ]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
word_model = Word2Vec(path)

In [ ]:
import gensim
word_model2 = gensim.models.Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  
#model = gensim.models.KeyedVectors.load('./model/GoogleNews-vectors-negative300.bin')

In [110]:
#words

In [ ]:
word_model["hello"]

In [113]:
import torch

from gensim.models import Word2Vec

word_model = Word2Vec(words, size=200, min_count=1, workers=4)

weights = torch.FloatTensor(word_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(weights)

In [51]:
embedding

Embedding(170, 200)

In [7]:
from gensim.models import KeyedVectors
filename='GoogleNews-vectors-negative300.bin'
word_model2 = KeyedVectors.load_word2vec_format(filename, binary=True)

In [8]:
weights = torch.FloatTensor(word_model2.wv.vectors)
embedding = nn.Embedding.from_pretrained(weights)

<ipython-input-8-9116ff7bd3c7>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  weights = torch.FloatTensor(word_model2.wv.vectors)


In [9]:
word_model2.wv["I", "hey"].shape

<ipython-input-9-e8632140b2a6>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  word_model2.wv["I", "hey"].shape


(2, 300)

In [12]:
len(word_model2.wv.vectors)

<ipython-input-12-a1086f84481e>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  len(word_model2.wv.vectors)


3000000

In [138]:
embedding

Embedding(3000000, 300)

## Train tagger

In [4]:
import torch.optim as optim
import torch
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=2, batch_size=1, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)
    tagger = FixedWindowTagger(vocab_words, vocab_tags, len(vocab_tags))
    tagger.model.to(device)
    
    optimizer = optim.Adam(tagger.model.parameters(), lr=lr)
    for i in range(n_epochs):
        total_loss = 0
        batch_nr = 0
        for x, y in training_examples_tagger(vocab_words, vocab_tags, train_data, tagger):
            x = x.to(device)
            y = y.to(device)
            batch_nr += 1
            
            optimizer.zero_grad()
            y_pred = tagger.model.forward(x)
            
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            total_loss += loss.item()
            optimizer.step()
            #if batch_nr % 500 == 1:
                #print(total_loss/batch_nr)
                #pass
        print(f"Epoch {i}, loss: {total_loss/batch_nr}")
    return tagger


In [5]:
tagger = train_fixed_window(train_data, n_epochs=10)

RuntimeError: CUDA out of memory. Tried to allocate 3.35 GiB (GPU 0; 2.95 GiB total capacity; 0 bytes already allocated; 1.71 GiB free; 0 bytes reserved in total by PyTorch)

In [ ]:
accuracy(tagger, dev_data)

In [7]:
accuracy(tagger, test_data)

NameError: name 'tagger' is not defined

In [13]:
accuracy(tagger, train_data)

0.9864651529925245